In [2]:
import ee
import geemap
import geopandas as gpd
import json

# ee.Authenticate()
ee.Initialize(project="mining-457808")

In [2]:
with open("C:\\space_technologies\\OpenPitMining\\niedzwedzia_gora\\niedzwiedzia_gora_polygon.geojson") as f:
    geojson_data = json.load(f)

polygon = geemap.geojson_to_ee(geojson_data)

image = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterBounds(polygon.geometry())
    .filterDate('2025-03-01', '2025-03-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
)

In [3]:
rectangle = polygon.geometry().bounds().buffer(500)

image = image.clip(rectangle)

rgb = image.select(['B4', 'B3', 'B2'])
ndvi = image.normalizedDifference(['B8', 'B4'])
bare_soil_mask = ndvi.lt(0.18)

ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
rgb_vis = {'min': 0, 'max': 3000}
polygon_style = {
    "color": "red",
    "fillColor": "00000000",
    "width": 2
}

map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(rgb, rgb_vis, "ortofotomapa")
map.add_layer(ndvi, ndvi_vis, "NDVI")
map.add_layer(polygon.style(**polygon_style), {}, "obszar górniczy")
map


Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…

In [4]:
bare_soil_mask = ndvi.lt(0.18)

bare_soil_vis = {'min': 0, 'max': 1, 'palette': ['black', 'white']}
mask_vis = {'min': 0, 'max': 1, 'palette': ['white', 'yellow']}

map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(rgb, rgb_vis, "ortofotomapa")
map.add_layer(ndvi, ndvi_vis, "NDVI")
map.add_layer(bare_soil_mask, bare_soil_vis, "Bare Soil Mask")
map.add_layer(polygon.style(**polygon_style), {}, "obszar górniczy")
map

Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
bare_soil = ndvi.lt(0.18)


ones_image = ee.Image.constant(1).clip(ndvi.geometry())

polygon_mask = ee.Image.constant(1).clip(polygon)

outside_mask_image = ones_image.where(polygon_mask.eq(1), 0)

result = bare_soil.updateMask(outside_mask_image)

map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(rgb, rgb_vis, "ortofotomapa")
map.add_layer(ndvi, ndvi_vis, "NDVI")
map.add_layer(bare_soil)
map.add_layer(ones_image)
map.add_layer(polygon_mask)
map.add_layer(outside_mask_image)
map.add_layer(result)

map.add_layer(polygon.style(**polygon_style), {}, "obszar górniczy")
map


Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…

In [21]:
# 1. Wektoryzacja masek odkrytej ziemi poza poligonem
bare_soil_vectors = result.reduceToVectors(
    geometryType='polygon',
    geometry=ndvi.geometry(),
    scale=10,  # rozdzielczość Sentinel-2
    maxPixels=1e8
)

# 2. Przefiltruj: tylko te obiekty, które przecinają granicę poligonu
bare_soil_touching = bare_soil_vectors.filterBounds(polygon)

# 3. (opcjonalnie) Rasteryzuj z powrotem, żeby pokazać na mapie jako maskę
# Najpierw przypisz sztuczną wartość do każdego wektora
bare_soil_touching = bare_soil_touching.map(lambda f: f.set('presence', 1))

# Rasteryzuj na podstawie tej właściwości
bare_soil_touching_raster = bare_soil_touching.reduceToImage(
    properties=['presence'],
    reducer=ee.Reducer.first()
)

vis_params = {'palette': ['orange'], 'opacity': 0.8}

map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(bare_soil_touching_raster, vis_params, "Plamy styczne z poligonem")
map.add_layer(polygon.style(**polygon_style), {}, "obszar górniczy")
map




Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…